### 简介
此版本为综合v1中两种方法采集到的所有可用信息作为数据集。

输入主要包含：
1. 脸部图片
2. 眼部部分截图
3. 提取出的脸部信息（左右宽度比，上下高度比）

输出：鼠标位置的横、纵坐标

主要更新内容：
路径保存问题，时间戳命名，csv内容

注意事项：
1. 当前版本中，通过摄像头采集照片，和采集照片后进行加工处理的两部分必须挨着完成，即不能从其他途径引入命名格式不同，或获取时间不同的图片（由于采用仅存于内存中的数据结构保存关键数据）

In [1]:
#引入所有需要的模块，如有未安装的库建议pip install
# PyAutoGUI是一个纯Python的GUI自动化工具，其目的是可以用程序自动控制鼠标和键盘操作，多平台支持（Windows，OS X，Linux）。
#cv2即opencv2，用于实现电脑摄像头的调用等操作
#PIL 为Python 常用图像处理模块
import pyautogui as pag
import time
import requests
import os
import cv2
import urllib3,base64
import json
from PIL import Image
from urllib.parse import urlencode
import csv

###  一、收集并保存摄像头照片
通过openCV调用摄像头，**在按下空格键时保存照片**，命名为鼠标所在位置的元组形式。

运行此单元格后将弹出名为Capture的摄像头视频页面，敲击空格即可保存图片，通过ESC键终止运行。

In [2]:
#初始化相关数据文件夹和暂存用数据结构
paths = ['./dataset_origin/','./dataset_face/','./dataset_eye/']
for path in paths:
    if not os.path.exists(path):
        os.makedirs(path)
#存放图片时间戳（字符串形式，固定为10位）和对应鼠标坐标（元组形式）
photoDic = {}
# 存放csv文件的原始数据，每行五列，对应图片名（时间戳），额外参数（paramX,paramY),鼠标横、纵坐标
rows = []

In [3]:
#获取屏幕大小
screenWidth,screenHeight = pag.size()
#计算获得屏幕原点位置，定为屏幕中心点
origin = (screenWidth/2,screenHeight/2)

cap=cv2.VideoCapture(0) #调用摄像头，0为电脑自带摄像头，1为外部摄像头
while(1):
    ret,frame = cap.read()
    k=cv2.waitKey(1)
    if k==27: #Esc键退出
        break
    elif k==32:#空格键保存图片
        #获取当前鼠标绝对位置
        currMouseX,currMouseY = pag.position() 
        #将鼠标绝对位置转化为相对于原点的相对坐标，并进行归一化处理
        #对归一化处理有疑惑可参考：https://www.jianshu.com/p/95a8f035c86c
        currMouse = ((currMouseX-origin[0])/origin[0],(origin[1]-currMouseY)/origin[1])
        #调用python自带的库time，快捷地获取当前时间生成时间戳
        curr_time = str(int(time.time()))
        #将图片的命名信息和对应鼠标坐标暂时保存至此字典中
        photoDic[curr_time] = currMouse
        cv2.imwrite(paths[0]+ curr_time +'.jpg',frame)
        print(curr_time) #方便调试，若成功保存则会立刻在控制台输出
    cv2.imshow("capture", frame)
cap.release()
cv2.destroyAllWindows()

1613915168
1613915169
1613915170


### 二、调用人脸识别API获取关键点信息
调用百度AI-人脸识别以照片进行处理，并找出照片中的脸部关键点信息，以方便对图片信息进行下一步的处理。

BaiduAI 相关技术文档：https://ai.baidu.com/ai-doc/FACE/yk37c1u4t

注：此方法仅支持2QPS的查询率（即每秒2次的查询率），如需实际应用或用于实时演示则需更改此部分的代码或购买更大的处理能力。也可通过申请多个百度账号，建立账号池以增大qps。

In [4]:
# 首先通过百度AI提供的方法获取申请服务时的ID参数access_token，此参数至少一个月需更新一次，此处提供的为我的access_token
import requests 

# client_id 为官网获取的AK， client_secret 为官网获取的SK
host = 'https://aip.baidubce.com/oauth/2.0/token?grant_type=client_credentials&client_id=QGsRMBXewEr9zcmapc3HeIVC&client_secret=PloEXIBs9Sf30tYVVtOppftGGgnqTwNh'
access_token = 'wait_to_get'
response = requests.get(host)
if response:
    access_token = response.json()['access_token']
    print(access_token)

#BaiduAI 人脸关键点识别方法调用，参数为文件路径，返回json格式的BaiduAI的反馈
def BaiduMethod(filepath):
    file = open(filepath,'rb')
    img =  open(filepath,'rb')
    #参数images：图像base64编码 分别base64编码后的2张图片数据，需urlencode，半角逗号分隔，单次请求最大不超过20M
    img1 = base64.b64encode(img.read())
    request_url = "https://aip.baidubce.com/rest/2.0/face/v3/detect"+ "?access_token=" + access_token
    params = {'image':str(img1,'utf-8'),'image_type':'BASE64','face_field':'landmark'}
    headers = {'content-type': 'application/json'}
    return requests.post(request_url, data=params, headers=headers)

24.c16e32ee9bc4d45c81b5326d79b9709c.2592000.1616507179.282335-22192901


In [29]:
#测试用，可忽略
#此处改为所想读取的单个图片
filepath = paths[0]+'1613652963.jpg'
#考虑到百度AI平台的对并发量的限制，此处推荐分开处理，即获取response后不再重复对同一张照片进行处理
response = BaiduMethod(filepath)
if response:
    print (response.json())

{'error_code': 0, 'error_msg': 'SUCCESS', 'log_id': 3520115101891, 'timestamp': 1613653149, 'cached': 0, 'result': {'face_num': 1, 'face_list': [{'face_token': 'a85ff30e1579e8ff63932726564c7acb', 'location': {'left': 282.39, 'top': 164.44, 'width': 202, 'height': 207, 'rotation': -5}, 'face_probability': 1, 'angle': {'yaw': 8.32, 'pitch': 4.3, 'roll': -8.22}, 'landmark': [{'x': 335.96, 'y': 194.67}, {'x': 426.79, 'y': 186.54}, {'x': 382.05, 'y': 237.5}, {'x': 388.93, 'y': 298.03}], 'landmark72': [{'x': 286.38, 'y': 208.36}, {'x': 291.3, 'y': 240.12}, {'x': 299.09, 'y': 271.96}, {'x': 308.82, 'y': 303.3}, {'x': 329.66, 'y': 334.42}, {'x': 361.85, 'y': 357.45}, {'x': 395.06, 'y': 362.52}, {'x': 427.94, 'y': 352.67}, {'x': 458.79, 'y': 326.53}, {'x': 477.9, 'y': 292.18}, {'x': 484.29, 'y': 258.36}, {'x': 487.15, 'y': 224.6}, {'x': 487.68, 'y': 191.39}, {'x': 315.44, 'y': 200.37}, {'x': 323.8, 'y': 191.81}, {'x': 334.38, 'y': 188.37}, {'x': 345.34, 'y': 189.96}, {'x': 355.21, 'y': 198.1}, 

<center>关键点识别图片</center>
<img src='https://ai.bdstatic.com/file/52BC00FFD4754A6298D977EDAD033DA0' width = '75%'>

### 三、根据已有信息生成数据集
数据集格式：
图片命名均为时间戳
1. ./dataset_origin/ 存放摄像头保存的原始照片
2. ./dataset_face/ 存放整张脸的照片，命名为当前时间戳
3. ./dataset_eye/ 存放眼部照片，截取的范围可进一步精确
4. ./dataset_csv 存放图片解析的额外信息(左右宽度比，上下高度比）和图片输出的鼠标横纵坐标

In [5]:
#定义图片加工方法
#处理dataset_origin中的原始图片，得到相同命名，存放路径不同的脸部和眼部图片。
def imgCrop(landmark,img):
    box = (landmark[0]['x'],landmark[24]['y'],landmark[12]['x'],landmark[6]['y'])#截取脸部图片
    box2 = (landmark[22]['x'],landmark[22]['y'],landmark[22]['x']+150,landmark[22]['y']+50) #截取眼部图片
    img2 = img.crop(box)
    img3 = img.crop(box2)
    #     img2.show() #实时显示切割图片方便微调
    img2.save(paths[1]+file) # 存放脸部帐篷
    img3.save(paths[2]+file) #存放眼部照片
    
#定义图片解析方法
def imgAnalyse(landmark,file):
#      参数一：左半脸与右半脸宽度之比（相对观察者而言的左右）
    paramX = abs(landmark[0]['x']-landmark[57]['x'])/abs(landmark[12]['x']-landmark[57]['x'])
#       参数二：鼻尖与眼部上侧坐标纵坐标差值(建议进行进一步改进)
    eyeY = (landmark[15]['y'] + landmark[32]['y'])/2
    paramY = eyeY - landmark[57]['y']
    return (paramX,paramY)

In [6]:
#遍历原始照片照片文件夹，调用 baidu 方法进行人脸分析，并根据反馈信息进行加工处理
dirs = os.listdir(paths[0])
for file in dirs:
    if(file=='.ipynb_checkpoints'):
        continue
    filepath = os.path.join(paths[0],file)
#     filepath = os.getcwd()+'\dataset_orgin\'+file
    #调用BaiduAi 方法获取json包
    response = BaiduMethod(filepath) 
    if response and response.json()['error_code']==0:#成功收到且未报错
        landmark = response.json()['result']['face_list'][0]['landmark72']
        img = Image.open(filepath)
        imgCrop(landmark,img)
        param = imgAnalyse(landmark,file)
        # 一行数据格式为：文件名，额外参数，鼠标坐标
        fileName = file[0:10] 
        mouse = photoDic[fileName]
        rows.append([fileName,param[0],param[1],mouse[0],mouse[1]])

#将获得的数据写入csv文件中
headers = ['fileName','paramX','paramY','mouseX','mouseY']
with open('dataset.csv','w') as f:
    f_csv = csv.writer(f)
    f_csv.writerow(headers)
    f_csv.writerows(rows)